Transformation is done based on this document -- [Link](https://docs.google.com/document/d/1_wt-wO18sa5iQBy3b-WpSHDjCqRl3MeN67N1v3dTZtQ/edit#heading=h.usgit3agfa62)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib

In [2]:
#Commented out until a shapefile for HP village boundaries can be acquired
#bharat_maps_villages = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Maps\HP\DDD_HP_villageboundary.geojson')

In [3]:
mission_antyodaya_df = pd.read_excel(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\HP_antyodaya_village_dataset.xlsx')

In [4]:
mission_antyodaya_df.head()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,total_no_of_farmers_add_fert_in_soil_as_per_report,total_no_of_elected_representatives,total_no_of_elect_rep_oriented_under_rgsa,total_no_of_elect_rep_undergone_training_under_rgsa,total_approved_labour_budget_for_year,total_expenditure_approved_under_nrm_labour_budget_during_yr,total_area_covered_under_irrigation_using_drip_sprinkler,total_hhd_having_piped_water_connection,village_latitude,village_longitude
0,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10183,BHUMTI,...,9,1,0,0,2.0,0.0,0.0,44,31.117117,76.924280
1,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10183,BHUMTI,...,0,0,0,0,0.0,0.0,0.0,1,31.117117,76.924280
2,2,HIMACHAL PRADESH,18,KANGRA,109,Dhira,164,SULAH,8836,KAHANPAT,...,0,1,0,0,50000.0,50000.0,0.0,21,32.018017,76.459785
3,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10205,PALOG,...,0,0,0,0,0.0,0.0,0.0,4,31.084871,77.033690
4,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10205,PALOG,...,0,0,0,0,0.0,0.0,0.0,18,31.084877,77.033676


In [5]:
mission_antyodaya_df['sub_district_code'].value_counts()

sub_district_code
181     482
165     481
139     398
184     395
143     332
       ... 
6953     16
186      14
6886     10
193       9
7048      2
Name: count, Length: 172, dtype: int64

In [6]:
mission_antyodaya_edited = mission_antyodaya_df[['sub_district_code','district_name', 'sub_district_name', 'net_sown_area_in_hac', 
                             'total_hhd', 'total_hhd_having_piped_water_connection', 
                             'total_hhd_not_having_sanitary_latrines', 
                             'availablility_hours_of_domestic_electricity', 
                             'availability_of_telephone_services']]

# Convert to numeric columns that should be summed or averaged
columns_to_sum = ['net_sown_area_in_hac', 'total_hhd', 'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']
columns_to_avg = ['availablility_hours_of_domestic_electricity', 'availability_of_telephone_services']

mission_antyodaya_edited[columns_to_sum] = mission_antyodaya_edited[columns_to_sum].apply(pd.to_numeric, errors='coerce')
mission_antyodaya_edited[columns_to_avg] = mission_antyodaya_edited[columns_to_avg].apply(pd.to_numeric, errors='coerce')

# Group by 'sub_district_code' and merge accordingly
aggregated_df = mission_antyodaya_edited.groupby('sub_district_code').agg({
    'sub_district_name': 'first',
    'district_name': 'first',
    'net_sown_area_in_hac': 'sum',
    'total_hhd': 'sum',
    'total_hhd_having_piped_water_connection': 'sum',
    'total_hhd_not_having_sanitary_latrines': 'sum',
    'availablility_hours_of_domestic_electricity': 'mean',
    'availability_of_telephone_services': 'mean'
}).reset_index()

aggregated_df.head()

C:\Users\saura\AppData\Local\Temp\ipykernel_19652\3933786443.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mission_antyodaya_edited[columns_to_sum] = mission_antyodaya_edited[columns_to_sum].apply(pd.to_numeric, errors='coerce')
C:\Users\saura\AppData\Local\Temp\ipykernel_19652\3933786443.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mission_antyodaya_edited[columns_to_avg] = mission_antyodaya_edited[columns_to_avg].apply(pd.to_numeric, errors='coerce')


,sub_district_code,sub_district_name,district_name,net_sown_area_in_hac,total_hhd,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines,availablility_hours_of_domestic_electricity,availability_of_telephone_services
0,83,Pangi,CHAMBA,27219.770,6617,1195,1049,2.754717,3.632075
1,84,Chaurah,CHAMBA,20584.072,20801,5738,1506,4.100334,2.979933
2,85,Saluni,CHAMBA,1929.490,11175,2908,124,4.015267,2.396947
3,86,Bhalai,CHAMBA,2899.230,6639,3274,228,4.012821,2.256410
4,87,Dalhousie,CHAMBA,1570.300,10934,4414,570,3.974576,2.415254


In [7]:
aggregated_df.to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya/MissionAntyodaya2020_HP_vul.csv', index=False)

In [8]:
mission_antyodaya_geometry = [
        Point(xy) for xy in zip(
            mission_antyodaya_df.village_longitude,
            mission_antyodaya_df.village_latitude
        )
    ]

In [9]:
gdf_points = GeoDataFrame(mission_antyodaya_df, geometry=mission_antyodaya_geometry)

In [11]:
gdf_polygons = GeoDataFrame.from_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\HP temp\flood-data-ecosystem-Himachal-Pradesh\Maps\bharatmaps_HP_subdistricts.geojson')

In [12]:
gdf_polygons

,objectid,stcode11,dtcode11,sdtcode11,stname,dtname,sdtname,subdt_lgd,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape),geometry
0,1,02,023,00088,HIMACHAL PRADESH,Chamba,Bhattiyat,88,16,2,0.0,0.0,,5,2,3.695661e+08,101900.165772,"MULTIPOLYGON (((76.03658 32.52185, 76.03717 32..."
1,2,02,023,00092,HIMACHAL PRADESH,Chamba,Bharmour,92,16,2,0.0,0.0,,2,3,1.334739e+09,239400.256000,"MULTIPOLYGON (((76.78355 32.54782, 76.78401 32..."
2,3,02,023,00090,HIMACHAL PRADESH,Chamba,Chamba,90,16,2,0.0,0.0,,3,4,1.422634e+09,195066.808024,"MULTIPOLYGON (((76.09595 32.70255, 76.09641 32..."
3,4,02,023,00084,HIMACHAL PRADESH,Chamba,Churah,84,16,2,0.0,0.0,,1,5,1.652463e+09,224217.320127,"MULTIPOLYGON (((76.22226 33.02277, 76.22303 33..."
4,5,02,023,00087,HIMACHAL PRADESH,Chamba,Dalhousie,87,16,2,0.0,0.0,,4,6,2.287591e+08,85680.411712,"MULTIPOLYGON (((75.92925 32.61495, 75.93061 32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,3317,02,033,00183,HIMACHAL PRADESH,Shimla,Junga,183,23,2,0.0,0.0,,62,3318,1.504535e+08,63973.266994,"MULTIPOLYGON (((77.30567 31.00461, 77.30567 31..."
119,3318,02,032,00176,HIMACHAL PRADESH,Sirmaur,Ronhat,176,24,2,0.0,0.0,,59,3319,1.341459e+08,63052.419007,"MULTIPOLYGON (((77.62574 30.80987, 77.62637 30..."
120,3396,02,027,00130,HIMACHAL PRADESH,Mandi,Bhadrota,130,22,2,0.0,0.0,,35,3397,1.077063e+08,58588.320611,"MULTIPOLYGON (((76.77496 31.62199, 76.77486 31..."
121,3397,02,029,00147,HIMACHAL PRADESH,Una,Bharwain,147,26,2,0.0,0.0,,41,3398,1.361356e+08,101878.868579,"MULTIPOLYGON (((76.06412 31.84618, 76.06429 31..."


In [13]:
()
gdf_polygons.rename(columns={'subdt_lgd': 'sub_district_code'}, inplace=True)
gdf_polygons.rename(columns={'sdtname': 'sub_district_name'}, inplace=True)
gdf_polygons.rename(columns={'dtname': 'district_name'}, inplace=True)

In [14]:
gdf_polygons

,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape),geometry
0,1,02,023,00088,HIMACHAL PRADESH,Chamba,Bhattiyat,88,16,2,0.0,0.0,,5,2,3.695661e+08,101900.165772,"MULTIPOLYGON (((76.03658 32.52185, 76.03717 32..."
1,2,02,023,00092,HIMACHAL PRADESH,Chamba,Bharmour,92,16,2,0.0,0.0,,2,3,1.334739e+09,239400.256000,"MULTIPOLYGON (((76.78355 32.54782, 76.78401 32..."
2,3,02,023,00090,HIMACHAL PRADESH,Chamba,Chamba,90,16,2,0.0,0.0,,3,4,1.422634e+09,195066.808024,"MULTIPOLYGON (((76.09595 32.70255, 76.09641 32..."
3,4,02,023,00084,HIMACHAL PRADESH,Chamba,Churah,84,16,2,0.0,0.0,,1,5,1.652463e+09,224217.320127,"MULTIPOLYGON (((76.22226 33.02277, 76.22303 33..."
4,5,02,023,00087,HIMACHAL PRADESH,Chamba,Dalhousie,87,16,2,0.0,0.0,,4,6,2.287591e+08,85680.411712,"MULTIPOLYGON (((75.92925 32.61495, 75.93061 32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,3317,02,033,00183,HIMACHAL PRADESH,Shimla,Junga,183,23,2,0.0,0.0,,62,3318,1.504535e+08,63973.266994,"MULTIPOLYGON (((77.30567 31.00461, 77.30567 31..."
119,3318,02,032,00176,HIMACHAL PRADESH,Sirmaur,Ronhat,176,24,2,0.0,0.0,,59,3319,1.341459e+08,63052.419007,"MULTIPOLYGON (((77.62574 30.80987, 77.62637 30..."
120,3396,02,027,00130,HIMACHAL PRADESH,Mandi,Bhadrota,130,22,2,0.0,0.0,,35,3397,1.077063e+08,58588.320611,"MULTIPOLYGON (((76.77496 31.62199, 76.77486 31..."
121,3397,02,029,00147,HIMACHAL PRADESH,Una,Bharwain,147,26,2,0.0,0.0,,41,3398,1.361356e+08,101878.868579,"MULTIPOLYGON (((76.06412 31.84618, 76.06429 31..."


In [15]:
gdf_points.crs = gdf_polygons.crs

In [16]:
result = gpd.sjoin(gdf_points, gdf_polygons, how="left", op="within")

d:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\.conda\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [ ]:
result.tail()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,subdt_lgd,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape)
19153,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8503,RIT (GUGGA),...,110.0,18.0,2.0,0.0,0.0,,13.0,2120.0,2.660055e+08,115337.042400
19154,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8503,RIT (GUGGA),...,110.0,18.0,2.0,0.0,0.0,,13.0,2120.0,2.660055e+08,115337.042400
19155,2,HIMACHAL PRADESH,18,KANGRA,6897,Chadiyar,157,LAMBAGAON,8491,KUDANG,...,112.0,18.0,2.0,0.0,0.0,,20.0,2111.0,4.089918e+08,128549.762319
19156,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8489,JANGAL,...,140.0,17.0,2.0,0.0,0.0,,37.0,2861.0,2.381732e+08,102588.129238
19157,2,HIMACHAL PRADESH,18,KANGRA,6900,Alampur,157,LAMBAGAON,8489,JANGAL,...,140.0,17.0,2.0,0.0,0.0,,37.0,2861.0,2.381732e+08,102588.129238


In [ ]:
result.shape

(19158, 217)

In [ ]:
'''
result[['gp_code','village_code','sub_district_code','sdtname','net_sown_area_in_hac','total_hhd',
        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',
       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(
    '/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam_vul.csv', index=False)

# Number of households in each urban area

In [21]:
# Number of households in each urban area
hp_urban_shapes = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\HP temp\flood-data-ecosystem-Himachal-Pradesh\Maps\DDD_hp_urban_shapes.geojson')
hp_urban_shapes

,pc11_s_id,pc11_d_id,sub_district_code,village_code,tv_name,geometry
0,02,023,0084,6782,R.F.Singra (364),"MULTIPOLYGON (((76.15731 32.97092, 76.15700 32..."
1,02,023,0084,6842,R.F.Shagwari (298),"MULTIPOLYGON (((76.02339 32.83204, 76.02411 32..."
2,02,023,0084,6846,Shagwari (296),"MULTIPOLYGON (((76.03999 32.82270, 76.03939 32..."
3,02,023,0084,6902,Daonri (408),"MULTIPOLYGON (((76.15012 32.85159, 76.14952 32..."
4,02,023,0084,6944,Dhar Maulgot (428),"MULTIPOLYGON (((76.26518 32.84914, 76.26485 32..."
...,...,...,...,...,...,...
1952,02,034,0199,27300,Bulcho Dhar (18),"MULTIPOLYGON (((78.14636 31.42011, 78.14726 31..."
1953,02,034,0199,27301,Dhar Pashel (28),"MULTIPOLYGON (((78.18658 31.43936, 78.18670 31..."
1954,02,034,0199,27302,Tesha Guden (27/3),"MULTIPOLYGON (((78.19781 31.42965, 78.19798 31..."
1955,02,034,0199,27304,D.P.F.C-152 (B) (31),"MULTIPOLYGON (((78.21413 31.43492, 78.21406 31..."


In [22]:

# TOTAL POPULATION IN EACH URBAN AREA
import rasterio
import rasterstats

worldpop_raster = rasterio.open(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\HP temp\flood-data-ecosystem-Himachal-Pradesh\Maps\hp_ppp_2020_UNadj.tif')
worldpop_raster_array = worldpop_raster.read(1)

sum_dicts = rasterstats.zonal_stats(hp_urban_shapes.to_crs(worldpop_raster.crs),
                                     worldpop_raster_array,
                                     affine= worldpop_raster.transform,
                                     stats= ['sum'],
                                     nodata=worldpop_raster.nodata,
                                     geojson_out = True)

dfs = []
for sd in sum_dicts:
    dfs.append(pd.DataFrame([sd['properties']]))

pop_zonal_stats_df = pd.concat(dfs).reset_index(drop=True)
pop_zonal_stats_df = pop_zonal_stats_df.rename(columns={'sum':'sum_population'})
# Divide population by 4.3 for households
pop_zonal_stats_df['urban_hhd'] = pop_zonal_stats_df['sum_population']/4.3
urban_hhd = pop_zonal_stats_df.groupby('sub_district_code')[['urban_hhd']].sum().reset_index()
urban_hhd['urban_electricity'] = urban_hhd['urban_hhd']*20
urban_hhd['urban_tele'] = urban_hhd['urban_hhd']*3
urban_hhd['urban_hhd_pipe'] = urban_hhd['urban_hhd']*0.1873
urban_hhd['urban_no_sanitation'] = urban_hhd['urban_hhd']*0.063

In [24]:
urban_hhd


,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation
0,0084,372.315463,7446.309254,1116.946388,69.734686,23.455874
1,0085,4141.186763,82823.735259,12423.560289,775.644281,260.894766
2,0086,35.736821,714.736415,107.210462,6.693507,2.251420
3,0087,1737.280685,34745.613701,5211.842055,325.392672,109.448683
4,0088,1564.596160,31291.923204,4693.788481,293.048861,98.569558
...,...,...,...,...,...,...
93,0192,2416.389188,48327.783758,7249.167564,452.589695,152.232519
94,0193,943.739899,18874.797972,2831.219696,176.762483,59.455614
95,0197,991.819976,19836.399518,2975.459928,185.767881,62.484658
96,0198,2676.533872,53530.677446,8029.601617,501.314794,168.621634


# Vul variables

In [23]:
vul_df = pd.read_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya/MissionAntyodaya2020_HP_vul.csv')

In [25]:
vul_df = vul_df.rename(columns={'total_hhd':'rural_hhd'})

## net_sown_area_in_hac

In [26]:
net_sown_area_in_hac_rc = vul_df.groupby(['sub_district_code'])[['net_sown_area_in_hac']].sum()

In [27]:
net_sown_area_in_hac_rc

,net_sown_area_in_hac
sub_district_code,
83,27219.770
84,20584.072
85,1929.490
86,2899.230
87,1570.300
...,...
6967,1341.610
6969,1213.287
6970,1003.190


## Rural - Urban Households in each rc

In [29]:
rural_hhd = vul_df.groupby('sub_district_code')[['rural_hhd']].sum().reset_index()
rural_hhd

,sub_district_code,rural_hhd
0,83,6617
1,84,20801
2,85,11175
3,86,6639
4,87,10934
...,...,...
167,6967,3387
168,6969,6192
169,6970,4429
170,7048,287


In [30]:
vul_df[vul_df.sub_district_code == 6919]

,sub_district_code,sub_district_name,district_name,net_sown_area_in_hac,rural_hhd,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines,availablility_hours_of_domestic_electricity,availability_of_telephone_services
137,6919,Majheen,KANGRA,868.02,3050,2009,251,3.666667,2.229167


In [34]:
urban_hhd['sub_district_code'] = urban_hhd['sub_district_code'].astype(int)

In [35]:
total_hhd = urban_hhd.merge(rural_hhd, how='outer', on='sub_district_code')
total_hhd

,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,rural_hhd
0,83,NaN,NaN,NaN,NaN,NaN,6617.0
1,84,372.315463,7446.309254,1116.946388,69.734686,23.455874,20801.0
2,85,4141.186763,82823.735259,12423.560289,775.644281,260.894766,11175.0
3,86,35.736821,714.736415,107.210462,6.693507,2.251420,6639.0
4,87,1737.280685,34745.613701,5211.842055,325.392672,109.448683,10934.0
...,...,...,...,...,...,...,...
168,6967,NaN,NaN,NaN,NaN,NaN,3387.0
169,6969,NaN,NaN,NaN,NaN,NaN,6192.0
170,6970,NaN,NaN,NaN,NaN,NaN,4429.0
171,7048,NaN,NaN,NaN,NaN,NaN,287.0


In [36]:
total_hhd = total_hhd.fillna(0)
total_hhd['total_hhd']=total_hhd['rural_hhd']+total_hhd['urban_hhd']

In [37]:
total_hhd

,sub_district_code,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,rural_hhd,total_hhd
0,83,0.000000,0.000000,0.000000,0.000000,0.000000,6617.0,6617.000000
1,84,372.315463,7446.309254,1116.946388,69.734686,23.455874,20801.0,21173.315463
2,85,4141.186763,82823.735259,12423.560289,775.644281,260.894766,11175.0,15316.186763
3,86,35.736821,714.736415,107.210462,6.693507,2.251420,6639.0,6674.736821
4,87,1737.280685,34745.613701,5211.842055,325.392672,109.448683,10934.0,12671.280685
...,...,...,...,...,...,...,...,...
168,6967,0.000000,0.000000,0.000000,0.000000,0.000000,3387.0,3387.000000
169,6969,0.000000,0.000000,0.000000,0.000000,0.000000,6192.0,6192.000000
170,6970,0.000000,0.000000,0.000000,0.000000,0.000000,4429.0,4429.000000
171,7048,0.000000,0.000000,0.000000,0.000000,0.000000,287.0,287.000000


## availablility_hours_of_domestic_electricity

In [38]:
# Map category to mean electricity available
electricity_dict = {1:2.5, 2:6, 3:10, 4:12, 5:0}

In [39]:
elect_df = vul_df.replace({"availablility_hours_of_domestic_electricity": electricity_dict})[['sub_district_code','rural_hhd','availablility_hours_of_domestic_electricity']]

In [40]:
elect_df['rural_electricity'] = elect_df['rural_hhd']*elect_df['availablility_hours_of_domestic_electricity']

In [43]:
elect_df = elect_df.groupby('sub_district_code')[['rural_hhd','rural_electricity']].sum().reset_index()

In [44]:
elect_df

,sub_district_code,rural_hhd,rural_electricity
0,83,6617,18227.962264
1,84,20801,85291.056856
2,85,11175,44870.610687
3,86,6639,26641.115385
4,87,10934,43458.016949
...,...,...,...
167,6967,3387,13096.400000
168,6969,6192,24684.324324
169,6970,4429,53148.000000
170,7048,287,3444.000000


In [45]:
sd_electricity = elect_df.merge(urban_hhd, on='sub_district_code', how='outer')


In [46]:
sd_electricity = sd_electricity.fillna(0)

In [47]:
sd_electricity['total_electricity'] = sd_electricity['rural_electricity'] + sd_electricity['urban_electricity']

In [49]:
sd_electricity['total_hhd'] = sd_electricity['rural_hhd'] + sd_electricity['urban_hhd']

In [50]:
sd_electricity

,sub_district_code,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd
0,83,6617.0,18227.962264,0.000000,0.000000,0.000000,0.000000,0.000000,18227.962264,6617.000000
1,84,20801.0,85291.056856,372.315463,7446.309254,1116.946388,69.734686,23.455874,92737.366110,21173.315463
2,85,11175.0,44870.610687,4141.186763,82823.735259,12423.560289,775.644281,260.894766,127694.345946,15316.186763
3,86,6639.0,26641.115385,35.736821,714.736415,107.210462,6.693507,2.251420,27355.851800,6674.736821
4,87,10934.0,43458.016949,1737.280685,34745.613701,5211.842055,325.392672,109.448683,78203.630651,12671.280685
...,...,...,...,...,...,...,...,...,...,...
168,6967,3387.0,13096.400000,0.000000,0.000000,0.000000,0.000000,0.000000,13096.400000,3387.000000
169,6969,6192.0,24684.324324,0.000000,0.000000,0.000000,0.000000,0.000000,24684.324324,6192.000000
170,6970,4429.0,53148.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53148.000000,4429.000000
171,7048,287.0,3444.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3444.000000,287.000000


In [51]:
sd_electricity['avg_electricity'] = sd_electricity['total_electricity']/sd_electricity['total_hhd']

In [52]:
sd_electricity

,sub_district_code,rural_hhd,rural_electricity,urban_hhd,urban_electricity,urban_tele,urban_hhd_pipe,urban_no_sanitation,total_electricity,total_hhd,avg_electricity
0,83,6617.0,18227.962264,0.000000,0.000000,0.000000,0.000000,0.000000,18227.962264,6617.000000,2.754717
1,84,20801.0,85291.056856,372.315463,7446.309254,1116.946388,69.734686,23.455874,92737.366110,21173.315463,4.379917
2,85,11175.0,44870.610687,4141.186763,82823.735259,12423.560289,775.644281,260.894766,127694.345946,15316.186763,8.337215
3,86,6639.0,26641.115385,35.736821,714.736415,107.210462,6.693507,2.251420,27355.851800,6674.736821,4.098417
4,87,10934.0,43458.016949,1737.280685,34745.613701,5211.842055,325.392672,109.448683,78203.630651,12671.280685,6.171723
...,...,...,...,...,...,...,...,...,...,...,...
168,6967,3387.0,13096.400000,0.000000,0.000000,0.000000,0.000000,0.000000,13096.400000,3387.000000,3.866667
169,6969,6192.0,24684.324324,0.000000,0.000000,0.000000,0.000000,0.000000,24684.324324,6192.000000,3.986486
170,6970,4429.0,53148.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53148.000000,4429.000000,12.000000
171,7048,287.0,3444.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3444.000000,287.000000,12.000000


## availability_of_telephone_services

In [53]:
telephone_df = vul_df.copy()

In [54]:
telephone_df['rural_tele'] = vul_df['rural_hhd']*vul_df['availability_of_telephone_services']

In [55]:
telephone_df = telephone_df[['sub_district_code', 'rural_hhd', 'rural_tele']]

In [56]:
telephone_df = telephone_df.groupby('sub_district_code')[['rural_hhd','rural_tele']].sum().reset_index()

In [57]:
sd_tele = telephone_df.merge(total_hhd[['sub_district_code', 'urban_hhd','urban_tele']], on='sub_district_code', how='outer')

In [58]:
sd_tele = sd_tele.fillna(0)

In [59]:
sd_tele['total_tele'] = sd_tele['rural_tele'] + sd_tele['urban_tele']

In [60]:
sd_tele['total_hhd'] = sd_tele['rural_hhd'] + sd_tele['urban_hhd']

In [61]:
sd_tele['avg_tele'] = round(sd_tele['total_tele']/sd_tele['total_hhd'])

In [62]:
sd_tele

,sub_district_code,rural_hhd,rural_tele,urban_hhd,urban_tele,total_tele,total_hhd,avg_tele
0,83,6617.0,24033.443396,0.000000,0.000000,24033.443396,6617.000000,4.0
1,84,20801.0,61985.588629,372.315463,1116.946388,63102.535017,21173.315463,3.0
2,85,11175.0,26785.877863,4141.186763,12423.560289,39209.438151,15316.186763,3.0
3,86,6639.0,14980.307692,35.736821,107.210462,15087.518155,6674.736821,2.0
4,87,10934.0,26408.389831,1737.280685,5211.842055,31620.231886,12671.280685,2.0
...,...,...,...,...,...,...,...,...
168,6967,3387.0,8213.475000,0.000000,0.000000,8213.475000,3387.000000,2.0
169,6969,6192.0,15145.297297,0.000000,0.000000,15145.297297,6192.000000,2.0
170,6970,4429.0,10376.514286,0.000000,0.000000,10376.514286,4429.000000,2.0
171,7048,287.0,717.500000,0.000000,0.000000,717.500000,287.000000,2.0


## total_hhd_having_piped_water_connection

In [64]:
pipe_df = vul_df.copy()

In [65]:
pipe_df = pipe_df.groupby('sub_district_code')[['total_hhd_having_piped_water_connection']].sum().reset_index()

In [66]:
pipe_df = pipe_df.merge(urban_hhd[['sub_district_code','urban_hhd_pipe']], on='sub_district_code', how='outer')

In [67]:
pipe_df = pipe_df.fillna(0)

In [69]:
pipe_df['sd_piped_hhds'] = + pipe_df['urban_hhd_pipe'] + pipe_df['total_hhd_having_piped_water_connection'] 

In [70]:
pipe_df = pipe_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [71]:
pipe_df['sd_piped_hhds_pct'] = 100*pipe_df['sd_piped_hhds']/pipe_df['total_hhd']

In [72]:
pipe_df

,sub_district_code,total_hhd_having_piped_water_connection,urban_hhd_pipe,sd_piped_hhds,total_hhd,sd_piped_hhds_pct
0,83,1195.0,0.000000,1195.000000,6617.000000,18.059544
1,84,5738.0,69.734686,5807.734686,21173.315463,27.429501
2,85,2908.0,775.644281,3683.644281,15316.186763,24.050662
3,86,3274.0,6.693507,3280.693507,6674.736821,49.150904
4,87,4414.0,325.392672,4739.392672,12671.280685,37.402633
...,...,...,...,...,...,...
168,6967,298.0,0.000000,298.000000,3387.000000,8.798347
169,6969,3028.0,0.000000,3028.000000,6192.000000,48.901809
170,6970,2367.0,0.000000,2367.000000,4429.000000,53.443215
171,7048,287.0,0.000000,287.000000,287.000000,100.000000


## total_hhd_not_having_sanitary_latrines

In [73]:
nosanitation_df = vul_df.copy()

In [74]:
nosanitation_df = nosanitation_df.groupby('sub_district_code')[['total_hhd_not_having_sanitary_latrines']].sum().reset_index()

In [75]:
nosanitation_df = nosanitation_df.merge(urban_hhd[['sub_district_code','urban_no_sanitation']], on='sub_district_code', how='outer')

In [76]:
nosanitation_df = nosanitation_df.fillna(0)

In [77]:
nosanitation_df['sd_nosanitation_hhds'] = nosanitation_df['urban_no_sanitation'] + nosanitation_df['total_hhd_not_having_sanitary_latrines'] 

In [78]:
nosanitation_df = nosanitation_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [79]:
nosanitation_df['sd_nosanitation_hhds_pct'] = 100*nosanitation_df['sd_nosanitation_hhds']/nosanitation_df['total_hhd']

In [80]:
nosanitation_df

,sub_district_code,total_hhd_not_having_sanitary_latrines,urban_no_sanitation,sd_nosanitation_hhds,total_hhd,sd_nosanitation_hhds_pct
0,83,1049.0,0.000000,1049.000000,6617.000000,15.853106
1,84,1506.0,23.455874,1529.455874,21173.315463,7.223507
2,85,124.0,260.894766,384.894766,15316.186763,2.512993
3,86,228.0,2.251420,230.251420,6674.736821,3.449595
4,87,570.0,109.448683,679.448683,12671.280685,5.362115
...,...,...,...,...,...,...
168,6967,166.0,0.000000,166.000000,3387.000000,4.901092
169,6969,186.0,0.000000,186.000000,6192.000000,3.003876
170,6970,160.0,0.000000,160.000000,4429.000000,3.612554
171,7048,1.0,0.000000,1.000000,287.000000,0.348432


# ANTYODAYA MASTER

In [81]:
antyodaya_master_df = net_sown_area_in_hac_rc.reset_index()

In [82]:
antyodaya_master_df = antyodaya_master_df.merge(sd_electricity[['sub_district_code', 'avg_electricity']], how='outer')

In [83]:
antyodaya_master_df = antyodaya_master_df.merge(sd_tele[['sub_district_code', 'avg_tele']])

In [84]:
antyodaya_master_df = antyodaya_master_df.merge(pipe_df[['sub_district_code', 'sd_piped_hhds_pct']])

In [85]:
antyodaya_master_df = antyodaya_master_df.merge(nosanitation_df[['sub_district_code','sd_nosanitation_hhds_pct']])


In [86]:
antyodaya_master_df

,sub_district_code,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
0,83,27219.770,2.754717,4.0,18.059544,15.853106
1,84,20584.072,4.379917,3.0,27.429501,7.223507
2,85,1929.490,8.337215,3.0,24.050662,2.512993
3,86,2899.230,4.098417,2.0,49.150904,3.449595
4,87,1570.300,6.171723,2.0,37.402633,5.362115
...,...,...,...,...,...,...
168,6967,1341.610,3.866667,2.0,8.798347,4.901092
169,6969,1213.287,3.986486,2.0,48.901809,3.003876
170,6970,1003.190,12.000000,2.0,53.443215,3.612554
171,7048,56.640,12.000000,2.0,100.000000,0.348432


In [87]:
antyodaya_master_df = pd.merge(gdf_polygons,antyodaya_master_df,on='sub_district_code', how='outer')

In [88]:
antyodaya_master_df[antyodaya_master_df.avg_electricity.isnull()]

,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,...,ac_no,test,st_area(shape),st_length(shape),geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct


In [89]:
antyodaya_master_df = antyodaya_master_df.merge(total_hhd[['sub_district_code', 'total_hhd']], on='sub_district_code')
antyodaya_master_df['total_hhd'] = antyodaya_master_df['total_hhd'].apply(lambda x: round(x,0))

In [90]:

sub_district_names_mapping = vul_df.set_index('sub_district_code')['sub_district_name'].to_dict()
# Map the sub_district_names back to the merged DataFrame
antyodaya_master_df['sub_district_name'] = antyodaya_master_df['sub_district_code'].map(sub_district_names_mapping)

# Check if there are any missing sub_district_names
missing_names = antyodaya_master_df['sub_district_name'].isna().sum()
print(f"Number of missing sub_district_names: {missing_names}")

Number of missing sub_district_names: 1


In [91]:

district_names_mapping = vul_df.set_index('sub_district_code')['district_name'].to_dict()
# Map the sub_district_names back to the merged DataFrame
antyodaya_master_df['district_name'] = antyodaya_master_df['sub_district_code'].map(district_names_mapping)

# Check if there are any missing sub_district_names
missing_names = antyodaya_master_df['district_name'].isna().sum()
print(f"Number of missing district_names: {missing_names}")

Number of missing district_names: 1


In [95]:
missing_names = antyodaya_master_df[antyodaya_master_df['district_name'].isna()]
missing_names


,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,...,test,st_area(shape),st_length(shape),geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct,total_hhd
99,3316.0,02,033,00182,HIMACHAL PRADESH,NaN,NaN,182,23.0,2.0,...,3317.0,2.384353e+07,31992.99194,"MULTIPOLYGON (((77.16427 31.11695, 77.16483 31...",NaN,20.0,3.0,18.73,6.3,42494.0


In [97]:
antyodaya_master_df = antyodaya_master_df.dropna()
antyodaya_export = antyodaya_master_df.drop(['geometry','st_area(shape)', 'st_length(shape)','dist_lgd','sdtcode11','dtcode11','remarks','ac_no' ,'test','state_lgd','st_area_sh','st_length_' , 'stcode11','stname', 'objectid'                         
                           ],axis=1)
antyodaya_export.round(2).to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\HP temp\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya\antyodaya_variables.csv', index=False)

In [93]:
antyodaya_export.shape

(173, 9)

In [ ]:
antyodaya_master_df

,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,...,test,st_area(shape),st_length(shape),geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct,total_hhd
0,6.0,02,023,00083,HIMACHAL PRADESH,CHAMBA,Pangi,83,16.0,2.0,...,7.0,2.154975e+09,227109.042007,"POLYGON ((76.57266 33.20633, 76.56888 33.20689...",27219.770,2.754717,4.0,18.059544,15.853106,6617
1,4.0,02,023,00084,HIMACHAL PRADESH,CHAMBA,Chaurah,84,16.0,2.0,...,5.0,1.652463e+09,224217.320127,"POLYGON ((76.22226 33.02277, 76.22149 33.02291...",20584.072,4.100334,3.0,27.585212,7.240037,20801
2,7.0,02,023,00085,HIMACHAL PRADESH,CHAMBA,Saluni,85,16.0,2.0,...,8.0,5.078111e+08,130735.805692,"POLYGON ((75.84641 32.92777, 75.84582 32.92819...",1929.490,4.015267,2.0,26.022371,1.109620,11175
3,3312.0,02,023,00086,HIMACHAL PRADESH,CHAMBA,Bhalai,86,16.0,2.0,...,3313.0,2.037403e+08,83175.890417,"POLYGON ((76.03918 32.66771, 76.03916 32.66772...",2899.230,4.012821,2.0,49.314656,3.434252,6639
4,5.0,02,023,00087,HIMACHAL PRADESH,CHAMBA,Dalhousie,87,16.0,2.0,...,6.0,2.287591e+08,85680.411712,"POLYGON ((75.92925 32.61495, 75.92705 32.61633...",1570.300,3.974576,2.0,40.369490,5.213097,10934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,NaN,NaN,NaN,NaN,NaN,SOLAN,Mamlig,6967,NaN,NaN,...,NaN,NaN,NaN,None,1341.610,3.866667,2.0,8.798347,4.901092,3387
168,NaN,NaN,NaN,NaN,NaN,MANDI,Makridi,6969,NaN,NaN,...,NaN,NaN,NaN,None,1213.287,3.986486,2.0,48.901809,3.003876,6192
169,NaN,NaN,NaN,NaN,NaN,MANDI,Mandap,6970,NaN,NaN,...,NaN,NaN,NaN,None,1003.190,12.000000,2.0,53.443215,3.612554,4429
170,NaN,NaN,NaN,NaN,NaN,KANGRA,Sulah,7048,NaN,NaN,...,NaN,NaN,NaN,None,56.640,12.000000,2.0,100.000000,0.348432,287
